In [1]:
# dependencies for the script
#!pip install matplotlib pandas h5py lxml
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from dask import dataframe as dd 

In [4]:
import dask
#dask.set_options(get=dask.threaded.get)  # overwrite default with threaded scheduler
dask.set_options(get=dask.multiprocessing.get)  # overwrite default with multiprocessing scheduler

In [3]:
from dask.distributed import Client
client = Client()  # start distributed scheduler locally.  Launch dashboard
client

Client Scheduler: tcp://127.0.0.1:46048 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 64 Cores: 64 Memory: 405.47 GB


In [5]:
from pathlib import Path
import numpy as np
import pandas as pd
from DLCA_Reader import DLCA

/home/coria.fr/pouxa/DLCA-light/python-analysis/venv/lib64/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [6]:
%%time
# The folder with all .h5 and .xmf files
data_dir = Path("data/")
#data_dir = Path("data/")

# Read all data (~30s but the more data the longer)
metadata, times, Spheres, Aggregates = DLCA(data_dir).read(dask=True)

#Spheres = miSpheres.sort_values(["Time",'Label'])

CPU times: user 246 ms, sys: 79.6 ms, total: 325 ms
Wall time: 326 ms


In [29]:
# Data are well organized
# Looks a lot like an excel file
print(Spheres)

Dask DataFrame Structure:
                 Label     Posx     Posy     Posz   Radius
npartitions=100                                           
                 int64  float64  float64  float64  float64
                   ...      ...      ...      ...      ...
...                ...      ...      ...      ...      ...
                   ...      ...      ...      ...      ...
                   ...      ...      ...      ...      ...
Dask Name: read-hdf, 100 tasks


In [72]:
# Data are well organized
# Looks a lot like an excel file
print(Aggregates.compute())

               Time        Deltat            Dm  Np          Posx  \
Label                                                               
0      0.000000e+00  4.329293e-08  1.101216e-08   1  3.386984e-07   
0      1.167890e-07  4.329293e-08  1.101216e-08   1  3.386984e-07   
0      1.203820e-07  4.329293e-08  1.101216e-08   1  3.386984e-07   
0      1.296380e-07  4.329293e-08  1.101216e-08   1  3.386984e-07   
0      1.421640e-07  4.329293e-08  1.101216e-08   1  3.386984e-07   
0      1.479800e-07  4.329293e-08  1.101216e-08   1  3.386984e-07   
0      1.591620e-07  4.329293e-08  1.101216e-08   1  3.386984e-07   
0      1.640540e-07  4.329293e-08  1.101216e-08   1  3.386984e-07   
0      1.711360e-07  4.329293e-08  1.101216e-08   1  2.818241e-07   
0      1.760880e-07  4.329293e-08  1.101216e-08   1  2.818241e-07   
0      1.775370e-07  4.329293e-08  1.101216e-08   1  2.818241e-07   
0      1.921010e-07  4.329293e-08  1.101216e-08   1  2.078518e-07   
0      1.933150e-07  4.329293e-08 

In [19]:
# Create a new collumn in Spheres that contains the diameter of all spheres
Spheres["Diameter"] = 2 * Spheres["Radius"]

In [20]:
# Create a new collumn in Aggregates that contains the giration diameter of all aggregates
Aggregates["Dg"] = 2 * Aggregates["Rg"]

In [83]:
%%time 
# Create a new collumn in Aggregates that contains the mean diameter the spheres of each aggregates
# This also means that we need to compute
# the mean of the diameter of all the spheres that shares the same time and the same label
# Reminder : label is the unique number given to an aggregate to recognize it

# This kind of operation is also a litle bit long (few seconds)
Spheres_diam = Spheres.groupby(by=["Time", "Label"])["Diameter"].mean().reset_index(0).set_index("Label")

CPU times: user 76.9 ms, sys: 17.5 ms, total: 94.4 ms
Wall time: 70.9 ms


In [87]:
Aggregates["MeanDp"] = Spheres_diam["Diameter"]

In [88]:
# Compute Dg over Dp
Aggregates["DgOverDp"] = Aggregates["Dg"] / Aggregates["MeanDp"]

In [18]:
%%time 

# The same result could have been obtained with this one-liner (a litle bit faster)
Aggregates["DgOverDp"] = Aggregates["Rg"] / Spheres.groupby(by=["Time", "Label"])["Radius"].mean().reset_index(0)

ValueError: Not all divisions are known, can't align partitions. Please use `set_index` to set the index.

In [90]:
print(Aggregates["DgOverDp"].groupby(by="Time").agg((np.min,np.mean,np.max)))

Dask DataFrame Structure:
                  Time   Deltat       Dm     Np     Posx     Posy     Posz       Rg     Rmax  Surface   Volume      lpm       Dg   MeanDP   MeanDp DgOverDp
npartitions=2                                                                                                                                              
0              float64  float64  float64  int32  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64  float64
74                 ...      ...      ...    ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...
99                 ...      ...      ...    ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...      ...
Dask Name: assign, 63 tasks


KeyError: 'Time'

In [ ]:
# Rename DgOverDp for ease of use
Data = Aggregates["Dm"] 

In [ ]:
%matplotlib notebook
# this line was only for jupyter
import matplotlib.pyplot as plt

plt.figure()
plt.plot(Data.groupby("Time").mean())
plt.show()

In [ ]:
from scipy.optimize import minimize, minimize_scalar
from scipy.integrate import simps
def fit(datax, datay):
   
    def lognorm(x, s, loc):
        #loc = datax[maxi]
        pre = (np.sqrt(2*np.pi) * s)
        a = -0.5 * ((np.log(x/loc)) / s)**2 
        f= np.exp(a) / pre
        return f/f.sum()
    
    def cost(param):
        f = lognorm(datax, *param)
        err = np.sqrt(simps((f - datay)**2))
        return err
    
    maxi = np.argmax(datay)
    
    x0 = [0.5, datax[maxi]]
    res = minimize(cost, x0, bounds=((None,None),(datax[0],datax[-1])),options={"maxiter":1000})
    param = res.x
    #res = minimize_scalar(cost)
    
    #print(datay.std(),datay.mean())
    #print(x0, res.x)
    #print(res.message)
    res = lognorm(datax, *param)

    return res, param
    

In [ ]:
def graph(bins, debut, fin):
    
    data = Data.loc[debut:fin, :]
    
    hist, bin_edges = np.histogram(data, bins=bins)
    hist_norm = hist / hist.sum()
    hist_cum = np.cumsum(hist_norm)
    #ax.bar(bin_edges[:-1], hist_cum, width=np.diff(bin_edges), ec="k", align="edge", alpha = 0.2,
    #       label="{:5.2e}s -> {:5.2e}s".format(debut, fin))
    x = (bin_edges[1:] +  bin_edges[:-1]) /2
    
    cfit, p = fit(x, hist_norm)
    ccfit = np.cumsum(cfit)

    #print(data.shape)
    #fit = stats.lognorm.fit(data.values)
    #print(fit)
    #cfit = stats.lognorm.pdf(x, *fit)
    #cfit = cfit / cfit.sum()
    #ccfit = stats.lognorm.cdf(x, *fit)
    axes[0].plot(x, hist_norm,'-o',
               label="{:5.2e}s -> {:5.2e}s ({} Nagg)".format(debut, fin, len(data)))
    axes[1].plot(x, hist_cum,'-o',
               label=r"$\alpha$ = {:5.2e}, $\mu$ = {:5.2e}".format(*p))
    axes[0].plot(x, cfit,'k--')
    axes[1].plot(x, ccfit,'k--')

    #Dg.loc[debut:fin, :].plot.hist(ax=ax,
    #                                 bins=bins,
    #                                 logy=True,
    #                                 logx=True,
    #                                 label="{:5.2e}s -> {:5.2e}s".format(debut, fin), cumulative=True)
    return x, hist_norm

In [ ]:
def find_nearest(array, value):
    idx = np.searchsorted(array.values, value, side="left")
    if idx > 0 and (idx == len(array) or abs(value - array.iloc[idx-1]) < abs(value - array.iloc[idx])):
        return idx-1
    return idx

In [ ]:
%matplotlib notebook
# this line was only for jupyter

import matplotlib
matplotlib.rcParams['text.usetex'] = False
matplotlib.rcParams['text.latex.unicode'] = False
import matplotlib.pyplot as plt
from scipy import stats

# Create a new figure (we need it to put all graphs on the same figure)
fig, axes = plt.subplots(2,1,sharex=True)

Nagg = Data.groupby(by="Time").count()
Nagg_cum = np.cumsum(Nagg)
    
# Split the time range in even parts
split = "Nagg"
if split=="linear":
    limits = np.linspace(times[0], times[-1]/3, 5)
elif split=="geometric":
    limits = np.geomspace(times[1], times[-1], 5)
elif split=="Nagg":

    limits = np.linspace(Nagg_cum.iloc[0], Nagg_cum.iloc[-1], 500)
    
    f = np.vectorize(lambda x : find_nearest(Nagg_cum, x))
    limits = times[f(limits)]

elif split=="Ndt":
    limits = np.linspace(0, len(times)-1, 5, dtype=int)
    limits = times[limits]

it = len(limits) - np.geomspace(1, len(limits)-1, 5, dtype=int)[::-1]
limits_inf = limits[it-1]
limits_sup = limits[it]
#limits_inf[-1] = limits[-2]
#limits_sup[-1] = times[-1]

# Create the bins
bins = np.geomspace(Data.min(), Data.max(),20)
#bins = np.linspace(Dg.min(), Dg.max(),20)

for debut, fin in zip(limits_inf, limits_sup):
    x, y = graph(bins, debut, fin)
        
#graph(bins, limits[0], limits[1])
#graph(bins, limits[-2], limits[-1])

#ax.set_yscale('log')
axes[0].set_xscale('log')
axes[1].set_xscale('log')
axes[0].legend()
axes[1].legend()
axes[0].grid()
axes[1].grid()
fig.subplots_adjust(hspace=0)
plt.show()

In [ ]:
import matplotlib.animation

# We can also make an animation

fig, ax = plt.subplots()

def animate(i):
    ax.cla()
    
    ax.set_ylim((1e-1, 1e7))
    ax.set_xlim((9e-1, 2e1))
    
    debut = limits[i]
    fin = limits[i+1]
    
    # if last image, go further to be sure not to miss anything 
    if i == len(limits) - 1:
        fin *= 2
    DgOverDp.loc[debut:fin, :].plot.hist(ax=ax,
                                         bins=bins,
                                         logy=True,
                                         logx=True)
    ax.set_title("{:5.2e}s -> {:5.2e}s".format(debut, fin))

ani = matplotlib.animation.FuncAnimation(fig, animate, frames=len(limits)-1,repeat=False)

plt.show()


In [ ]:
from DLCA_Drawer import draw

In [ ]:
def generator():
    while True:
        for time in times:
            Positions = np.stack((Spheres["Posx"].loc[0.,:],
                                  Spheres["Posy"].loc[0.,:],
                                  Spheres["Posz"].loc[0.,:]),axis=1)
            radius = np.array(Spheres["Radius"].loc[time,:])
            color = np.array(Spheres["Label"].loc[time,:])
            yield Positions, radius, color
    

In [ ]:
draw(generator())

In [ ]:
plt.savefig("test.png")

In [68]:
%load_ext line_profiler

In [7]:
%load_ext Cython
#%set_env CFLAGS="-Ofast -march=native -fvect-cost-model=cheap" \
#                "-fopenmp -Wno-unused-variable -Wno-cpp -Wno-maybe-uninitialized"

In [8]:
%%cython
# cython: language_level=3
# cython: initializedcheck=False
# cython: binding=True
# cython: nonecheck=False
# cython: boundscheck=False
# cython: wraparound=False
# distutils: extra_link_args = -fopenmp

import numpy as np
cimport numpy as np
from libc.math cimport sqrt

def c_cov1(const double[:]& X,
          const double[:]& Y,
          const double[:]& Z,
          const double[:]& R):
  
    cdef long points = X.shape[0]
    cdef double res
    cdef long n
    
    cdef Py_ssize_t i,j
    
    n = 0
    res = 0.
    for i in range(points):
        for j in range(i+1,points):
            d = 1. - sqrt((X[i]-X[j])**2. +
                          (Y[i]-Y[j])**2. +
                          (Z[i]-Z[j])**2.) / (R[i]+R[j])
            if d > 0:
                res += d
                n+= 1
    if n>0:
        return res/n
    else:
        return 0


In [9]:
%%cython
# cython: language_level=3
# cython: initializedcheck=False
# cython: binding=True
# cython: nonecheck=False
# cython: boundscheck=False
# cython: wraparound=False
# distutils: extra_link_args = -fopenmp

import numpy as np
cimport numpy as np
from libc.math cimport sqrt

def c_cov(const double[:, :]& X):
  
    cdef long points = X.shape[0]
    cdef double res
    cdef long n
    
    cdef Py_ssize_t i,j
    
    n = 0
    res = 0.
    for i in range(points):
        for j in range(i+1,points):
            d = 1. - sqrt((X[i,1]-X[j,1])**2. +
                          (X[i,2]-X[j,2])**2. +
                          (X[i,3]-X[j,3])**2.) / (X[i,4]+X[j,4])
            if d > 0:
                res += d
                n+= 1
    if n>0:
        return res/n
    else:
        return 0


In [7]:
from Analyze_cython import c_cov, c_cov1

In [12]:
def compute_cov(group):
    """
    Each group contains a number of spheres (x,y,z,r),
    I want to compute the mean coverage
    """

    n = len(group)

    # if only one sphere, no coverage
    if n == 1:
        return 0.

    # this statement alone cost 65% !
    #data = group.values
    X = group["Posx"].values
    Y = group["Posy"].values
    Z = group["Posz"].values
    R = group["Radius"].values

    # behind c_cov is a cython implementation of what is presented bellow
    # the cython code is invisible to cProfile, so it's fast enough
    return c_cov(X, Y, Z, R)

    # for two different spheres in the group
    X1, X2 = np.triu_indices_from(data.T, k=1)

    # renaming things for readability
    _, x1, y1, z1, r1 = data[X1].T
    _, x2, y2, z2, r2 = data[X2].T

    # my definition of coverage
    cov = 1 - np.sqrt((x1-x2)**2 + (y1-y2)**2 + (z1-z2)**2) / (r1+r2)

    # ignoring negative values (no contact)
    cov = cov[cov > 0]

    # Averaging
    if cov.size > 0:
        res = cov.mean()
    else:
        res = 0

    return res


In [13]:
%%time

#import cProfile

#%lprun -f compute_cov -f mycdist cov = Spheres.groupby(by=["Time", "Label"]).apply(compute_cov)
#%prun Spheres.groupby(by=["Time", "Label"]).apply(compute_cov)
#%timeit Spheres.groupby(by=["Time", "Label"]).apply(compute_cov)
Spheres1 = Spheres.reset_index(0)

cov = Spheres.groupby(by=["Time", "Label"]).apply(compute_cov, meta=("Cov","f8"))
cov1 = Spheres1.groupby(by=["Time", "Label"]).apply(compute_cov, meta=("Cov","f8"))

#cProfile.run('Spheres.groupby(by=["Time", "Label"]).apply(compute_cov)', "test.pstats")

CPU times: user 37.4 ms, sys: 129 ms, total: 167 ms
Wall time: 163 ms


In [31]:
%%time
res = cov.compute()

CPU times: user 5min 22s, sys: 1min 59s, total: 7min 22s
Wall time: 10min 48s


In [9]:
%%time
cov1.compute()

KeyboardInterrupt: 

In [ ]:
%%time
from dask.diagnostics import ProgressBar
with ProgressBar():
    cov1.compute()

[####                                    ] | 11% Completed |  1min 21.1s

In [11]:
%%time
from dask.diagnostics import ProgressBar
with ProgressBar():
    cov.compute()

[########################################] | 100% Completed | 10min 16.5s
CPU times: user 1min 17s, sys: 2min 44s, total: 4min 2s
Wall time: 10min 25s


In [16]:
import dask.array as da
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof, CacheProfiler() as cprof:
        cov.compute()

from dask.diagnostics import visualize
visualize([prof, rprof, cprof])


Column(id='c6801e5d-d46f-4d2c-ba9a-80574bc29f6d', ...)

In [13]:
%timeit cov.compute()

315 ms ± 19.6 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [14]:
import pandas as pd
simu = DLCA(Path("data_small/"))

In [15]:
simu.read_metadata()

In [20]:
print(Spheres)

              Label          Posx          Posy          Posz        Radius
Time     Num                                                               
0.000000 0        0  3.386984e-07  1.589845e-07  3.156847e-07  6.025496e-09
         1        1  3.675054e-07  7.963736e-08  1.351358e-07  5.888543e-09
         2        2  1.119772e-07  2.233176e-07  1.924494e-07  5.380718e-09
         3        3  1.470528e-07  2.069633e-07  3.838650e-07  6.802969e-09
         4        4  2.562696e-07  2.891584e-07  5.708315e-08  5.312195e-09
         5        5  6.571124e-09  9.791307e-08  5.532111e-08  6.053221e-09
         6        6  6.316083e-08  1.616296e-07  5.232142e-08  3.797439e-09
         7        7  4.026887e-07  8.798422e-08  2.067745e-07  6.237204e-09
         8        8  2.469688e-07  1.193369e-07  2.570115e-07  5.068430e-09
         9        9  1.989743e-07  3.921473e-07  1.179200e-07  5.902088e-09
         10      10  2.123426e-07  3.103694e-07  1.613411e-07  6.586051e-09
         11 

In [27]:
Spheres.to_hdf("data.h5", key="spheres", mode='w')
Aggregates.to_hdf("data.h5", key="aggregats", mode='a')

In [28]:
print(pd.read_hdf("data.h5", key="spheres"))


              Label          Posx          Posy          Posz        Radius
Time     Num                                                               
0.000000 0        0  3.386984e-07  1.589845e-07  3.156847e-07  6.025496e-09
         1        1  3.675054e-07  7.963736e-08  1.351358e-07  5.888543e-09
         2        2  1.119772e-07  2.233176e-07  1.924494e-07  5.380718e-09
         3        3  1.470528e-07  2.069633e-07  3.838650e-07  6.802969e-09
         4        4  2.562696e-07  2.891584e-07  5.708315e-08  5.312195e-09
         5        5  6.571124e-09  9.791307e-08  5.532111e-08  6.053221e-09
         6        6  6.316083e-08  1.616296e-07  5.232142e-08  3.797439e-09
         7        7  4.026887e-07  8.798422e-08  2.067745e-07  6.237204e-09
         8        8  2.469688e-07  1.193369e-07  2.570115e-07  5.068430e-09
         9        9  1.989743e-07  3.921473e-07  1.179200e-07  5.902088e-09
         10      10  2.123426e-07  3.103694e-07  1.613411e-07  6.586051e-09
         11 

In [29]:
print(pd.read_hdf("data.h5", key="aggregats"))

                      Deltat            Dm  Np          Posx          Posy  \
Time     Label                                                               
0.000000 0      4.329293e-08  1.101216e-08   1  3.386984e-07  1.589845e-07   
         1      4.134729e-08  1.088611e-08   1  3.675054e-07  7.963736e-08   
         2      3.452327e-08  1.040546e-08   1  1.119772e-07  2.233176e-07   
         3      5.518594e-08  1.170213e-08   1  1.470528e-07  2.069633e-07   
         4      3.364957e-08  1.033891e-08   1  2.562696e-07  2.891584e-07   
         5      4.369226e-08  1.103750e-08   1  6.571124e-09  9.791307e-08   
         6      1.719543e-08  8.739596e-09   1  6.316083e-08  1.616296e-07   
         7      4.638861e-08  1.120423e-08   1  4.026887e-07  8.798422e-08   
         8      3.063221e-08  1.009858e-08   1  2.469688e-07  1.193369e-07   
         9      4.153773e-08  1.089864e-08   1  1.989743e-07  3.921473e-07   
         10     5.172274e-08  1.151377e-08   1  2.123426e-07  3.